In [ ]:
import pandas as pd
import numpy as np
import gc
import datetime
import time

%matplotlib inline

In [ ]:
st = time.time()

adTrain = pd.read_csv('c:\\kaggle\\adtracking\\train.csv', skiprows=range(1,160000000))
#adData = pd.read_csv('c:\\kaggle\\adtracking\\train_sample.csv')

adTrain.info()
adTrain.head()

print(time.time()- st)

In [ ]:
st = time.time()

adTrain['click_time'] = pd.to_datetime(adTrain['click_time'])
adTrain['click_hour'] = adTrain['click_time'].dt.hour
adTrain.drop(['click_time','attributed_time'], axis=1, inplace=True)
gc.collect()
print(time.time()- st)

In [ ]:
import lightgbm as lgb

st = time.time()
X_train = adTrain[['ip', 'app', 'device', 'os', 'channel', 'click_hour']]
y_train = adTrain['is_attributed']

mdl = lgb.LGBMClassifier()
mdl.fit(X_train, y_train)
print(time.time()- st)

In [ ]:

st = time.time()

del adTrain
gc.collect()

adTest = pd.read_csv('c:\\kaggle\\adtracking\\train.csv', skiprows=range(1,50000000), nrows=10000000)

adTest['click_time'] = pd.to_datetime(adTest['click_time'])
adTest['click_hour'] = adTest['click_time'].dt.hour
adTest.drop(['click_time','attributed_time'], axis=1, inplace=True)
gc.collect()

print(time.time()- st)


In [ ]:
st = time.time()

X_test = adTest[['ip', 'app', 'device', 'os', 'channel', 'click_hour']]
y_test = adTest['is_attributed']

X_test_predict = mdl.predict_proba(X_test)
print(X_test_predict)
print(time.time()- st)

In [ ]:
import sklearn.metrics as metrics

auc_roc = metrics.roc_auc_score(y_test, X_test_predict[::, 1])

print(auc_roc)

In [ ]:
st = time.time()

del adTest
gc.collect()

adTest = pd.read_csv('c:\\kaggle\\adtracking\\test.csv')

adTest['click_time'] = pd.to_datetime(adTest['click_time'])
adTest['click_hour'] = adTest['click_time'].dt.hour
adTest.drop(['click_time'], axis=1, inplace=True)
gc.collect()

adTest.info()

print(time.time()- st)

In [ ]:
st = time.time()

ad_test = adTest[['ip', 'app', 'device', 'os', 'channel', 'click_hour']]

ad_test_predict = mdl.predict_proba(ad_test)
print(ad_test_predict)
print(time.time()- st)

In [ ]:
submit_file = 'c:\\kaggle\\adtracking\\Submission_lgbm.csv'
print(submit_file)
submission = pd.DataFrame({
        "click_id": adTest["click_id"],
        "is_attributed": ad_test_predict[::, 1]
    })

submission.to_csv(submit_file, index=False)